In [1]:
from torchvision import datasets
from matplotlib import pyplot as plt
import torch
from torchvision import transforms
import torch.optim as optim
import torch.nn as nn
from matplotlib import pyplot as plt
import numpy as np

data_path = 'C:/Users/keith/Desktop/School/RTAI/HW/HW4'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True) # <1>
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True) # <2>

Files already downloaded and verified
Files already downloaded and verified


In [2]:
transformed_cifar10 = datasets.CIFAR10(data_path, train=True, download=False,
                                      transform = transforms.Compose([
                                          transforms.ToTensor(),
                                          transforms.Normalize((0.4914, 0.4822, 0.4465),
                                                               (0.2470, 0.2435, 0.2616))
                                      ]))

transformed_cifar10_val = datasets.CIFAR10(data_path, train=False, download=False,
                                      transform = transforms.Compose([
                                          transforms.ToTensor(),
                                          transforms.Normalize((0.4914, 0.4822, 0.4465),
                                                               (0.2470, 0.2435, 0.2616))
                                      ]))

In [3]:
def softmax(x):
    return torch.exp(x) / torch.exp(x).sum()

In [4]:
label_map = {6: 0, 7: 1, 8: 2, 9: 3}
class_names = ['frog','horse','ship','truck']
cifar4 = [(img, label_map[label])
          for img, label in transformed_cifar10 
          if label in [6, 9]]
cifar4_val = [(img, label_map[label])
              for img, label in transformed_cifar10_val
              if label in [6, 9]]

In [5]:
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.fc2 = nn.Linear(32, 4)
        
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * 8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [6]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1): 
        loss_train = 0.0
        for imgs, labels in train_loader: 
            
            outputs = model(imgs)
            
            loss = loss_fn(outputs, labels) 

            optimizer.zero_grad()  
            
            loss.backward()
            
            optimizer.step()

            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))

In [7]:
train_loader = torch.utils.data.DataLoader(cifar4, batch_size=64, shuffle=True)
model = Net()
learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss()

import time

start = time.time()
training_loop(
    n_epochs = 200,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)
end = time.time()

ttime = end - start
print(str(ttime) + " seconds")

2021-04-21 17:45:39.486608 Epoch 1, Training loss 0.6763547581092567
2021-04-21 17:45:54.381015 Epoch 10, Training loss 0.20515698787702877
2021-04-21 17:46:10.127626 Epoch 20, Training loss 0.1202107252802249
2021-04-21 17:46:25.955342 Epoch 30, Training loss 0.08290823782752653
2021-04-21 17:46:41.752767 Epoch 40, Training loss 0.07055716647226719
2021-04-21 17:46:57.455806 Epoch 50, Training loss 0.053512479721384634
2021-04-21 17:47:13.268600 Epoch 60, Training loss 0.05071634403355776
2021-04-21 17:47:29.034498 Epoch 70, Training loss 0.04375589726297007
2021-04-21 17:47:44.714861 Epoch 80, Training loss 0.039451634265173965
2021-04-21 17:48:00.503049 Epoch 90, Training loss 0.03331922078475499
2021-04-21 17:48:16.246739 Epoch 100, Training loss 0.032592148228643376
2021-04-21 17:48:32.002179 Epoch 110, Training loss 0.026768304557379358
2021-04-21 17:48:48.251126 Epoch 120, Training loss 0.04254991645964788
2021-04-21 17:49:04.050123 Epoch 130, Training loss 0.021803904528682163


In [8]:
val_loader = torch.utils.data.DataLoader(cifar4_val, batch_size=64,
                                         shuffle=False)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():  # <1>
            for imgs, labels in loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <2>
                total += labels.shape[0]  # <3>
                correct += int((predicted == labels).sum())  # <4>

        print("Accuracy {}: {:.2f}".format(name , correct / total))

validate(model, train_loader, val_loader)

Accuracy train: 1.00
Accuracy val: 0.98


In [9]:
numel_list = [p.numel()
              for p in model.parameters()
              if p.requires_grad == True]
sum(numel_list), numel_list

(18156, [432, 16, 1152, 8, 16384, 32, 128, 4])